In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import random as rand 
import numpy as np

In [2]:
df = pd.read_csv("events.csv")
df["event_time"] = df["event_time"].str.removesuffix("UTC")
df["event_time"] = pd.to_datetime(df["event_time"])
df["time"] = df["event_time"].dt.time
df["month"] = df["event_time"].dt.month
df["day"] = df["event_time"].dt.day


In [3]:
set = df[(df["event_time"] >= "2021-02-01") & (df["event_time"]<"2021-02-15")]

In [4]:
for i in set.columns:
    print(f"a coluna {i} tem {len(set[set[i].isna() == True])}")

a coluna event_time tem 0
a coluna event_type tem 0
a coluna product_id tem 0
a coluna category_id tem 0
a coluna category_code tem 21363
a coluna brand tem 20361
a coluna price tem 0
a coluna user_id tem 0
a coluna user_session tem 3
a coluna time tem 0
a coluna month tem 0
a coluna day tem 0


In [5]:
teste = set.groupby(by =["user_id", "user_session", "event_type", "category_id","product_id"], as_index = False).agg(ini_sess = ("event_time", "min"), fin_sess = ("event_time", "max") )
teste["user_time"] = teste["fin_sess"] - teste["ini_sess"]
teste

,user_id,user_session,event_type,category_id,product_id,ini_sess,fin_sess,user_time
0,1515915625353234047,2bc2f4b2-831e-44f4-bc0e-efabe4ca5253,view,2144415924684325027,73593,2021-02-09 07:39:47,2021-02-09 07:43:05,0 days 00:03:18
1,1515915625353234047,2bc2f4b2-831e-44f4-bc0e-efabe4ca5253,view,2144415924684325027,1804643,2021-02-04 16:14:25,2021-02-08 07:34:55,3 days 15:20:30
2,1515915625353236157,jGh9mgZrOJ,view,2144415921001726020,886022,2021-02-04 04:19:46,2021-02-04 04:19:46,0 days 00:00:00
3,1515915625353236157,jGh9mgZrOJ,view,2144415921001726020,1547839,2021-02-11 15:50:10,2021-02-11 15:50:10,0 days 00:00:00
4,1515915625353236157,jGh9mgZrOJ,view,2144415926848585945,3721192,2021-02-11 16:10:01,2021-02-11 16:10:01,0 days 00:00:00
...,...,...,...,...,...,...,...,...
70286,1515915625605588306,XbXUhGj9Yn,view,2144415922427789416,1821813,2021-02-14 23:58:36,2021-02-14 23:58:36,0 days 00:00:00
70287,1515915625605588306,XbXUhGj9Yn,view,2144415922427789416,4154620,2021-02-14 23:59:00,2021-02-14 23:59:00,0 days 00:00:00
70288,1515915625605588405,NNc7dn3q2p,cart,2144415922427789416,4183864,2021-02-14 23:59:47,2021-02-14 23:59:47,0 days 00:00:00
70289,1515915625605588405,NNc7dn3q2p,view,2144415922427789416,4183864,2021-02-14 23:59:38,2021-02-14 23:59:38,0 days 00:00:00


In [6]:
set1 = pd.crosstab(teste["user_id"], teste["event_type"])
set1["purchase_npurchase"] = set1["purchase"] > 0 
set1["purchase_npurchase"] = ["purchase" if i == True else "non_purchase" for i in set1["purchase_npurchase"]]
set1["total_events"] = set1["cart"] + set1["purchase"] + set1["view"]
set1["unique_products"] = teste.groupby(by = "user_id")["product_id"].unique()
set1["unique_products"] = [len(i) for i in set1["unique_products"]]
set1["unique_categories"] = teste.groupby(by = "user_id")["category_id"].unique()
set1["unique_categories"] = [len(i) for i in set1["unique_categories"]]
set1["active_hours"] = teste.groupby("user_id")["fin_sess"].max().values - teste.groupby("user_id")["ini_sess"].min().values

set1["view_count"] = set1["view"]
set1["cart_count"] = set1["cart"]
set1.drop("view", axis = 1, inplace = True)
set1.drop("cart", axis = 1, inplace = True)

set1["session_count"] = teste.groupby(by = "user_id")["user_session"].unique()
set1["session_count"] = [len(i) for i in set1["session_count"]]
set1["av_session_duration"] = teste.groupby("user_id")["user_time"].mean()
set1["cart_event_ratio"] = set1["cart_count"]/set1["total_events"]
set1["carr_per_product"] = set1["cart_count"]/set1["unique_products"]
set1.drop("purchase", axis = 1, inplace = True)
set1.reset_index(inplace = True )
set1.drop("user_id", axis = 1, inplace = True)
set1["active_hours"] = set1["active_hours"].dt.total_seconds()/3600
set1["av_session_duration"] = set1["av_session_duration"].dt.total_seconds()/60
set1

event_type,purchase_npurchase,total_events,unique_products,unique_categories,active_hours,view_count,cart_count,session_count,av_session_duration,cart_event_ratio,carr_per_product
0,non_purchase,2,2,1,111.477778,2,0,1,2621.900000,0.000000,0.0
1,non_purchase,4,4,3,179.837500,4,0,1,0.000000,0.000000,0.0
2,non_purchase,15,13,4,199.203611,15,0,5,45.814444,0.000000,0.0
3,non_purchase,1,1,1,168.010000,1,0,1,10080.600000,0.000000,0.0
4,non_purchase,1,1,1,271.352222,1,0,1,16281.133333,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
39824,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.0
39825,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.0
39826,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.0
39827,non_purchase,2,2,1,0.006667,2,0,1,0.000000,0.000000,0.0


In [7]:
# balanceamento das amostras em 5:1 na proporção de non_purchase e purchase 

purchase = set1[set1["purchase_npurchase"] == "purchase"].copy()
indice = set1[set1["purchase_npurchase"] == "non_purchase"].index.tolist()
tent = rand.sample(indice, 2228*4)
n_purchase = set1.loc[tent].copy()
setfinal = pd.concat([purchase, n_purchase])
setfinal.reset_index(drop = True, inplace = True)
setfinal


event_type,purchase_npurchase,total_events,unique_products,unique_categories,active_hours,view_count,cart_count,session_count,av_session_duration,cart_event_ratio,carr_per_product
0,purchase,5,2,1,0.435556,3,1,2,2.136667,0.200000,0.500000
1,purchase,4,1,1,145.733056,2,1,2,0.000000,0.250000,1.000000
2,purchase,8,2,1,6.345833,4,2,3,6.031250,0.250000,1.000000
3,purchase,16,8,1,168.172778,9,5,2,655.203125,0.312500,0.625000
4,purchase,7,5,2,1.065556,5,1,1,0.000000,0.142857,0.200000
...,...,...,...,...,...,...,...,...,...,...,...
11135,non_purchase,1,1,1,0.014444,1,0,1,0.866667,0.000000,0.000000
11136,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.000000
11137,non_purchase,4,3,1,0.072222,3,1,1,0.000000,0.250000,0.333333
11138,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.000000


In [9]:
# separação das amostras em sets de treino e sets de teste, serão 10 sets no total, 3 de treino e 7 de teste 
lista = list([] for i in range(0,10))
rotulo = setfinal["purchase_npurchase"].unique()
dict = {c: setfinal[setfinal["purchase_npurchase"]== c ].index.values for c in rotulo}

for c in dict:
    np.random.shuffle(dict[c])
    for idx, i in enumerate(dict[c]):
        lista[i%10].append(i)



In [10]:
# verificação simples da extratificação
ver = setfinal.loc[lista[8]].copy()
print( f"""a quantidade de amostras no set 0 é:{len(ver)}\n
      a quantidade de amostras purchase é {len(ver[ver["purchase_npurchase"] == "purchase"] ) } """ )

a quantidade de amostras no set 0 é:1114

      a quantidade de amostras purchase é 222 


In [11]:
# separação em set de treino e set de teste 

lista_treino = list()
lista_teste = list()
treino = rand.sample(list(range(0,10)),7)
for i in treino:
    lista_treino.append(lista[i])

for i in range(0,10):
    if i not in treino:
        lista_teste.append(lista[i])

lista_treino = np.hstack([i for i in lista_treino])
lista_teste = np.hstack([i for i in lista_teste])

In [12]:
grupo_teste = [rand.choice(lista_treino) for i in range(len(lista_treino))]

In [13]:
teste = setfinal.loc[grupo_teste]
teste


event_type,purchase_npurchase,total_events,unique_products,unique_categories,active_hours,view_count,cart_count,session_count,av_session_duration,cart_event_ratio,carr_per_product
563,purchase,17,11,4,0.512778,11,4,1,3.110784,0.235294,0.363636
11063,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.000000
443,purchase,6,2,1,9.754167,4,1,3,0.569444,0.166667,0.500000
6679,non_purchase,2,2,1,0.025556,2,0,1,0.000000,0.000000,0.000000
3649,non_purchase,3,3,1,0.034167,3,0,1,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
10974,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.000000
5029,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.000000
9643,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.000000
2862,non_purchase,1,1,1,0.000000,1,0,1,0.000000,0.000000,0.000000


In [14]:
# cálculo de Gini_pai
p1  = len(teste[teste["purchase_npurchase"] == "purchase"])/len(teste)
p0  = len(teste[teste["purchase_npurchase"] == "non_purchase"])/len(teste)

Gini_pai = 1 - (p1**2 + p0**2)
Gini_pai


0.3334945051706657

In [ ]:
p0

0.8002051808155938

In [15]:
# cálculo de Gini para total events <= 2.5
# Gini esquerda 
Esquerda = teste[teste["total_events"] < 2.5]["purchase_npurchase"]
p0_es = (Esquerda == "non_purchase").astype(float).sum()/len(Esquerda)
p1_es = (Esquerda == "purchase").astype(float).sum()/len(Esquerda)
Gini_es = 1- (p0_es**2 + p1_es**2)

#gini_direita
Direita = teste[teste["total_events"] >= 2.5]["purchase_npurchase"]
p0_di = (Direita == "non_purchase").astype(float).sum()/len(Direita)
p1_di = (Direita == "purchase").astype(float).sum()/len(Direita)
Gini_di = 1- (p0_di**2 + p1_di**2)

Gini_pond = Gini_es*(len(Esquerda)/len(teste)) + Gini_di*(len(Direita)/len(teste))

Gini_pond


0.13119266557789633

In [157]:
# ===========================================================================================
# função pra calcular o Gini_pai 
def ginipai(set, var):

    lista = set[var].value_counts(normalize = True)
    
    Gini_pai = 1 - np.sum(lista**2)
    
    return(Gini_pai)

# ==========================================================================================
# função de cálculo do Gini do nó 
def ginino(set, var_grupos, var_sep, grupo_pos, limiar):
    
    # proporções de esqueda e direita
    Esquerda = set[set[var_sep] < limiar][var_grupos]
    Direita = set[set[var_sep] >=limiar][var_grupos]
    #if len(Esquerda) <= 0 or len(Direita) <=0:
        

    # Definindo o Gini da esquerda 
    p0_es = (Esquerda != grupo_pos).astype(float).sum()/len(Esquerda)
    p1_es = (Esquerda == grupo_pos).astype(float).sum()/len(Esquerda)
    Gini_es = 1- (p0_es**2 + p1_es**2)

    # Definindo o Gini da direita 
    p0_di = (Direita != grupo_pos).astype(float).sum()/len(Direita)
    p1_di = (Direita == grupo_pos).astype(float).sum()/len(Direita)
    Gini_di = 1- (p0_di**2 + p1_di**2)
    
    Gini_pond = Gini_es*(len(Esquerda)/len(set)) + Gini_di*(len(Direita)/len(set))

    return(Gini_pond)

# ========================================================================================
# Função de sorteio de variáveis e teste de repetidos limiares, deve ser usada em cada nó
def splitFunc(set, var, g_pos):

    
    Gini_Pai = ginipai(set, var)
    if Gini_Pai == 0:
        leaf = {"type": "leaf","class_dist": {0:len(set[set[var]!= g_pos]), 1:len(set[set[var] == g_pos])}, "Gini": Gini_Pai}
        return(leaf)
    ver1 = set[var].value_counts()
    if (ver1<=1).astype(float).sum() >=1:
        leaf = {"type": "leaf","class_dist": {0:len(set[set[var]!= g_pos]), 1:len(set[set[var] == g_pos])}, "Gini": Gini_Pai}
        return(leaf)
    
    vars = set.columns.tolist()
    vars.remove(var)
    nsort = (len(vars)**0.5)//1
    vars_sorteadas = rand.sample(vars, int(nsort))
    
    var_opt = None
    x_opt = None 
    absol = Gini_Pai
    for i in vars_sorteadas:
        valores = np.sort(set[i].unique())
        dict_var = {i: [(valores[j] + valores[j+1])/2 for j in range(0,len(valores)-1)]}
        for j in dict_var[i]:
            res = ginino(set, var, i, g_pos, j)
            if res < absol: 
                absol = res 
                var_opt = i
                x_opt = j

    if absol >= Gini_Pai:
        leaf = {"type": "leaf", "class_dist": {0:len(set[set[var]!= g_pos]), 1:len(set[set[var] == g_pos])}, "Gini": Gini_Pai }
        return(leaf)
    
    no_es = set[set[var_opt] < x_opt]
    no_di = set[set[var_opt] >= x_opt]

    splitFunc(no_es, var, g_pos)
    splitFunc(no_di, var, g_pos)

    node = {"type": "node", "variable":var_opt, "optmal":x_opt, "left_child": no_es, "right_child":no_di, "Gini": absol}
    return(node)
           




In [143]:
ver = np.array([1,2,3,4,5,0,1,0,1,2,3,5,8,1,0,8,9,1,0,1,2,3,1,0,1,2,1,0,5,4])
(ver<=1)

array([ True, False, False, False, False,  True,  True,  True,  True,
       False, False, False, False,  True,  True, False, False,  True,
        True,  True, False, False,  True,  True,  True, False,  True,
        True, False, False])

In [63]:
vars = set.columns.tolist()
vars.remove(var)

nsort = (len(vars)**0.5)//1
vars

['total_events',
 'unique_products',
 'unique_categories',
 'active_hours',
 'view_count',
 'cart_count',
 'session_count',
 'av_session_duration',
 'cart_event_ratio',
 'carr_per_product']

In [88]:
vars = set.columns.tolist()
vars.remove(var)
nsort = (len(vars)**0.5)//1
vars_sorteadas = rand.sample(vars, int(nsort))


for i in vars_sorteadas:
    valores = np.sort(set[i].unique())
    dict_var = {i: [(valores[j] + valores[j+1])/2 for j in range(0,len(valores)-1)]}
    for j in dict_var[i]:
        res = ginino(set, var, i, "purchase", j)
        print(res)


0.3208613652059239
0.32498144128655526
0.32599882876973324
0.329171498198729
0.32919922080475766
0.33139768021430277
0.3312578567197903
0.33205758273541347
0.33253692570042437
0.33269662467619193
0.33317547578670303
0.20779124202987048
0.13119266557789633
0.21316242838951624
0.25611843771325254
0.275444903059391
0.2874727472420105
0.2939040902884932
0.3012073496308143
0.3076727413619696
0.3117871454981766
0.3149341565349129
0.3167699885504452
0.319535167762752
0.3207657830706025
0.3215023970656307
0.32351158484687254
0.32528841767441796
0.325919048665822
0.32573393873016826
0.3257017762545496
0.3272825029498748
0.32772605209786665
0.32804072177315946
0.32882353711884227
0.3289793365511662
0.32876407219239084
0.3290800446056077
0.32962897582127065
0.3301063942419322
0.33026523342336883
0.33042389980838305
0.3310564431240851
0.3312139134890618
0.33152777785829735
0.3316840527877875
0.331485917309183
0.3318032785721996
0.33211951890899144
0.3322174047458271
0.3323771857288986
0.3326966246

In [40]:
# teste da função ginipai
ginipai(teste, "purchase_npurchase")

0.3334945051706657

In [51]:
for i in valores:
    ginino(teste, "purchase_npurchase", "cart_count", "purchase", i)
    print(50*"=")

0.042651637133219955 5963
0.2851036090549339 1835
0.09970464668391824
0.29096444582437875 7436
0.13804523671438607 362
0.2838656059041662
0.3173581797195959 7659
0.0693545882718285 139
0.3129374950297729
0.3249991126625791 7723
0.07679999999999998 75
0.32261197064543445
0.32895122037842484 7757
0.09280190362879237 41
0.3277096043247271
0.329754669627653 7765
0.058769513314967714 33
0.328607899922816
0.3307914186164329 7775
0.0 23
0.3298157578536504
0.33138226022825634 7780
0.0 18
0.33061733580095337
0.3317359054973903 7783
0.0 15
0.3310977882131558
0.33185364423187624 7784
0.0 14
0.3312578567197903
0.33208890742860864 7786
0.0 12
0.3315778703820399
0.33232388521816847 7788
0.0 10
0.33189771968185383
0.33267581752200637 7791
0.0 7
0.3323771857288986
0.3329100833087315 7793
0.0 5
0.33269662467619193
0.3332609492284133 7796
0.0 2
0.33317547578670303


In [158]:
splitFunc(teste, "purchase_npurchase", "purchase")

{'type': 'node',
 'variable': 'total_events',
 'optmal': 2.5,
 'left_child': event_type purchase_npurchase  total_events  unique_products  \
 11063            non_purchase             1                1   
 6679             non_purchase             2                2   
 6828             non_purchase             1                1   
 3434             non_purchase             2                1   
 3413             non_purchase             1                1   
 ...                       ...           ...              ...   
 10974            non_purchase             1                1   
 5029             non_purchase             1                1   
 9643             non_purchase             1                1   
 2862             non_purchase             1                1   
 9172             non_purchase             1                1   
 
 event_type  unique_categories  active_hours  view_count  cart_count  \
 11063                       1      0.000000           1           0  